# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from PIL import ImageGrab
import cv2
import time
from mss import mss

#mouse control and window dimensions
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 1024
import win32api, win32con
#Player Control
click_cooldown = 0.5

## Env setup

In [2]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017' # 1
#MODEL_NAME = 'ssd_inception_v2_coco_11_06_2017' # 2
MODEL_NAME = 'vayne_inference_graph'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

#NUM_CLASSES = 90
NUM_CLASSES = 1

## Download Model

In [5]:
#opener = urllib.request.URLopener()
#opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
#tar_file = tarfile.open(MODEL_FILE)
#for file in tar_file.getmembers():
#  file_name = os.path.basename(file.name)
#  if 'frozen_inference_graph.pb' in file_name:
#    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [222]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)#
def click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y,0,0)
def find_player_xy(boxes):
    playerwidth =  (boxes[3] - boxes[1])
    playerheight = boxes[2] - boxes[0]
    playerpos_y = (boxes[0] + playerheight/1.3)
    playerpos_x = (boxes[1] + playerwidth/2)
    return([playerpos_y, playerpos_x, playerheight, playerwidth])
def click_state(state_x, state_y, state_width, state_height, player_origin):
    x_click = (player_origin[0]/800)*SCREEN_WIDTH + state_x*(state_width/800)*SCREEN_WIDTH
    y_click = (player_origin[1]/450)*SCREEN_HEIGHT + (-1*state_y*(state_height/450)*SCREEN_HEIGHT)
    print(x_click)
    click(int(x_click), int(y_click))

In [224]:
sct = mss()

mon = {'top': 0, 'left': 0, 'width': SCREEN_WIDTH, 'height': SCREEN_HEIGHT}
last_time = time.time()
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      print('Loop took: {} seconds'.format(time.time()-last_time))
      last_time = time.time()
      #screen = cv2.resize(grab_screen(region=(0,40,1024,768)), (800,450))
      #screen = np.array(ImageGrab.grab(bbox=(0,40,1024,768)))
        
      sct.get_pixels(mon)
      screen = Image.frombytes('RGB', (sct.width, sct.height), sct.image)
      screen = np.array(screen)
      screen = cv2.resize(screen, (800,450))
        
      image_np = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})                  
      
        
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=4)

      #detect vayne
      gameover = False
      vaynefound = False
      for i,b in enumerate(boxes[0]):
          if scores[0][i] >= 0.6:
              if classes[0][i] == 1 and vaynefound == False:
                  vaynedound = True
                  player_position = find_player_xy(boxes[0][i])
                  #click(int(player_position[1]*SCREEN_WIDTH), int(player_position[0]*SCREEN_HEIGHT))
                  #draw a circle where the character is
                  player_origin = [int(player_position[1]*800), int(player_position[0]*450)]
                  cv2.circle(image_np, (player_origin[0], player_origin[1]), 2, (0, 0, 255), 2)
                  #calculate the origin of the center rectangle under the character
                  origin = [int(player_origin[0]+9)-int(player_position[2]*800/3), player_origin[1]-15,int(player_origin[0])+20, int(player_origin[1])+18]        
                  cv2.rectangle(image_np,(origin[0], origin[1]),(origin[2], origin[3]), (0, 255, 0), 1)
                  #create draw states
                  w = origin[2] - origin[0]
                  h = origin[3] - origin[1]
                  grid_x = int(SCREEN_WIDTH/w)
                  grid_y = int(SCREEN_HEIGHT/h)
                  for x in range(grid_x):
                        for y in range(grid_y):
                            #cv2.rectangle(image_np,(origin[0]+w*x, origin[1]+h*y),(origin[2]+w*x, origin[3]+h*y), (0, 255, 0), 1)
                            cv2.rectangle(image_np,(origin[0]+w*x-int(grid_x/2)*w, origin[1]+h*y-int(grid_y/2)*h),(origin[2]+w*x-int(grid_x/2)*w, origin[3]+h*y-int(grid_y/2)*h), (0, 255, 0), 1)
                  click_state(1,1,w,h, player_origin)
        
      cv2.imshow('window',image_np)
      if (cv2.waitKey(25) & 0xFF == ord('q')):
          cv2.destroyAllWindows()
          break
      if gameover == True:
          print('gameover')
          cv2.destroyAllWindows()
          break;


Loop took: 0.0035076141357421875 seconds
Loop took: 1.8827548027038574 seconds
Loop took: 0.39003705978393555 seconds
745.5999999999999
Loop took: 0.42226552963256836 seconds
748.8
Loop took: 0.437974214553833 seconds
Loop took: 0.37600135803222656 seconds
814.4
Loop took: 0.40152502059936523 seconds
827.1999999999999
Loop took: 0.37618279457092285 seconds
857.6
Loop took: 0.40543246269226074 seconds
Loop took: 0.4060797691345215 seconds
Loop took: 0.3749980926513672 seconds
932.8
Loop took: 0.40624499320983887 seconds
Loop took: 0.37599968910217285 seconds
988.8
Loop took: 0.405106782913208 seconds
Loop took: 0.40709948539733887 seconds
635.1999999999999
Loop took: 0.42162275314331055 seconds
Loop took: 0.37499213218688965 seconds
Loop took: 0.3582489490509033 seconds
628.8000000000001
Loop took: 0.437664270401001 seconds
636.8000000000001
915.1999999999999
Loop took: 0.4388444423675537 seconds
Loop took: 0.3737154006958008 seconds
1430.4
Loop took: 0.405078649520874 seconds
Loop took

616.0
Loop took: 0.39003777503967285 seconds
646.4
Loop took: 0.39003753662109375 seconds
Loop took: 0.3594551086425781 seconds
635.1999999999999
Loop took: 0.3910403251647949 seconds
638.4
864.0000000000001
Loop took: 0.4055790901184082 seconds
Loop took: 0.3439171314239502 seconds
635.2
312.0
Loop took: 0.41761064529418945 seconds
619.2
Loop took: 0.39003753662109375 seconds
631.9999999999999
Loop took: 0.40758442878723145 seconds
640.0
704.0
331.2
Loop took: 0.40407299995422363 seconds
614.4000000000001
697.6
332.8
Loop took: 0.42111968994140625 seconds
616.0
696.0
328.0
Loop took: 0.42261338233947754 seconds
617.6
692.8000000000001
328.0
Loop took: 0.39003682136535645 seconds
619.2
694.4
326.4
Loop took: 0.39053940773010254 seconds
617.6
694.4
337.59999999999997
Loop took: 0.4060788154602051 seconds
619.2
696.0
342.4
Loop took: 0.4065825939178467 seconds
619.2
691.2
Loop took: 0.42162108421325684 seconds
611.2
696.0
Loop took: 0.39003705978393555 seconds
619.2
694.4000000000001
345

636.8000000000001
Loop took: 0.3429133892059326 seconds
Loop took: 0.3599569797515869 seconds
633.6
406.4
Loop took: 0.3895139694213867 seconds
363.20000000000005
Loop took: 0.35945630073547363 seconds
361.6
620.8000000000001
Loop took: 0.40758419036865234 seconds
627.2
Loop took: 0.38953542709350586 seconds
659.2
Loop took: 0.3900322914123535 seconds
620.8000000000001
Loop took: 0.3744938373565674 seconds
632.0
323.20000000000005
Loop took: 0.37600159645080566 seconds
614.4
Loop took: 0.3749964237213135 seconds
632.0
Loop took: 0.3744969367980957 seconds
625.6
323.20000000000005
Loop took: 0.35945630073547363 seconds
633.6
323.20000000000005
Loop took: 0.3594553470611572 seconds
628.8000000000001
323.20000000000005
Loop took: 0.3744971752166748 seconds
633.6
Loop took: 0.3910400867462158 seconds
635.1999999999999
323.20000000000005
Loop took: 0.39069628715515137 seconds
624.0
Loop took: 0.39053869247436523 seconds
643.2
326.4
Loop took: 0.4201164245605469 seconds
619.2
323.20000000000

624.0
475.2
1315.2
Loop took: 0.4211232662200928 seconds
624.0
379.2
Loop took: 0.4055776596069336 seconds
347.2
628.8
Loop took: 0.39053964614868164 seconds
366.4
1273.6
Loop took: 0.37599921226501465 seconds
636.8
326.4
Loop took: 0.3739950656890869 seconds
235.20000000000002
635.1999999999999
619.2
Loop took: 0.3925435543060303 seconds
630.3999999999999
267.2
636.8
Loop took: 0.4050784111022949 seconds
736.0
643.1999999999999
1318.4
Loop took: 0.4376637935638428 seconds
Loop took: 0.3739948272705078 seconds
748.8000000000001
Loop took: 0.35845422744750977 seconds
224.0
595.2
Loop took: 0.3918130397796631 seconds
228.8
Loop took: 0.3749997615814209 seconds
Loop took: 0.3739943504333496 seconds
Loop took: 0.35844922065734863 seconds
620.8000000000001
838.4
Loop took: 0.39053964614868164 seconds
638.4000000000001
915.2
499.20000000000005
Loop took: 0.3898453712463379 seconds
507.20000000000005
1091.2
638.4000000000001
Loop took: 0.4226217269897461 seconds
996.8
Loop took: 0.39003705978

Loop took: 0.357952356338501 seconds
464.0
628.8
Loop took: 0.42112016677856445 seconds
286.4
Loop took: 0.39053869247436523 seconds
Loop took: 0.4065816402435303 seconds
873.6
1096.0
Loop took: 0.3739950656890869 seconds
972.8
633.6
1070.4
Loop took: 0.406583309173584 seconds
1083.2
948.8
280.0
Loop took: 0.3905367851257324 seconds
764.8
Loop took: 0.406583309173584 seconds
897.6
624.0
Loop took: 0.38753175735473633 seconds
848.0
635.1999999999999
Loop took: 0.36046457290649414 seconds
756.8
356.8
Loop took: 0.3900313377380371 seconds
684.8
Loop took: 0.34441518783569336 seconds
648.0
Loop took: 0.3744964599609375 seconds
640.0
846.4
Loop took: 0.3744974136352539 seconds
633.6
635.2
260.79999999999995
124.80000000000001
Loop took: 0.45270419120788574 seconds
265.6
548.8000000000001
632.0
Loop took: 0.39003705978393555 seconds
681.6
123.20000000000002
Loop took: 0.40658092498779297 seconds
280.0
Loop took: 0.35845255851745605 seconds
625.6
Loop took: 0.39045238494873047 seconds
700.8
L

707.1999999999999
Loop took: 0.46938180923461914 seconds
542.4
Loop took: 0.42112040519714355 seconds
505.6
644.8000000000001
Loop took: 0.43811988830566406 seconds
516.8
Loop took: 0.4371626377105713 seconds
515.2
Loop took: 0.3749983310699463 seconds
540.8
Loop took: 0.3739943504333496 seconds
545.6
Loop took: 0.3915410041809082 seconds
1204.8
547.1999999999999
Loop took: 0.4060795307159424 seconds
1203.2
641.5999999999999
Loop took: 0.3896975517272949 seconds
Loop took: 0.37549805641174316 seconds
662.4000000000001
Loop took: 0.3599565029144287 seconds
643.1999999999999
Loop took: 0.4050729274749756 seconds
558.4
Loop took: 0.3749980926513672 seconds
524.8
Loop took: 0.4050765037536621 seconds
638.4
505.6
Loop took: 0.39054036140441895 seconds
545.6
668.8
Loop took: 0.3599565029144287 seconds
643.1999999999999
523.2
Loop took: 0.374997615814209 seconds
534.4
Loop took: 0.39003729820251465 seconds
524.8000000000001
Loop took: 0.35895657539367676 seconds
528.0
Loop took: 0.40607786178

KeyboardInterrupt: 